# Process 进程

**1s = 1000 ms**  
**1 ms = 1000 us**  
**1 us = 1000 ns**  

### fork 函数


> fork 是通过 写时拷贝(copy-on-write)

>源文件共享只读 -> 需要写文件时拷贝    
 
>需要写的资源 -> 自己 的虚拟地址空间

# 进程间通信


### 同一个主机




#### 匿名管道（PIPE)

- 管道----一个内核中的缓冲器，粗存能力有限，有文件的特质->读/写操作

- [^匿名管道] 没有 文件实体 ， [^有名管道]有文件实体
  - 单向的  
  - 一次性的

> 原理

<p>不同进程之间，共享<b>虚拟内存空间</b>中的文件描述符(fd[1024])</p>


        #include<unistd.h>
        int pipe(int pipefd[2]);

#### 管道的特点

- 使用管道时要注意一下集中特殊情况

  1. 所有的指向管道写端的文件描述符都被关闭了（管道写端饮用计数0），进程读数据，管道读完后，再次读read会返回0 和EOF一样；

  2. 如果有指向管道写端的fd没有关闭（>0)，而持有者也没有写数据，读端完一次后再次read  会阻塞，知道写端再次写入才会继续；

  3. 如果所有指向管道读端的文件描述符都关闭了（水管只进不出），这时候向其中写入，会受到一个信号SIGPIPE，通常进程会异常终止；

  4. 如果有指向管道读端的文件描述符没有关闭（读>0)，也没有读数据，内容没有变少。此时有人写入，如果一直写入，写满后会阻塞，直到有空位置;

>总结
>- 读管道
> - 管道中 有数据: read返回实际读到字节
> - 管道中 无数据： 
>
>              写端被全部关闭：read返回 0 （相当读到文件末尾）
>              写端没有全部关闭： read  阻塞等待
>- 写管道   
> - 读端全部关闭： 异常终止（进程收到SIGPIPE信号）
> - 读端没有完全关闭：
>
>              管道已满： write阻塞
>              管道未满： 写入并返回实际写入的数字
                
  

### 有名管道（FIFO）
>First In First Out

### 内存映射 (memory-map)

- 注意事项
 1. 可以对返回的ptr ++ -- 操作， 但是 munmap不行 ， 需要保存原地址

 2. prot的权限 <= open的权限
 
 3. 如果文件偏移量为1000会怎么样？
 
        必须是1024的整数倍，返回 MAP_FAILED

4. mmap什么情况下会调用失败？

    - 第二个参数 length = 0
    - 第三个参数 prot 
    
        - 只给了写权限
        - prot PROT_READ | PROT_WRITE  oepn制定了 只读或只写 O_RDONLY/O_WRONLY
        

5. 可以open的时候O_CREAT 一个新文件来创建映射区吗？
    - 可以，但文件大小为0，无法映射
    - 可以对文件进行扩展

        lseek()
        truncate()

6. mmap关闭文件描述符，对mmap有影响吗？

        int fd = open("XX");
        mmap(,,,,fd,0);
        close(fd);

    - 映射区还存在，fd被关闭，没有任何影响

7. 对ptr 越界操作会怎么样（野地址）？

        void * ptr = mmap(NULL,100,,,,,);
        
        越界操作是非法的内存>>段错误




### 信号 SIGNAL
